# Common Size Balance Sheet

**Purpose**: The purpose of this notebook is to obtain the Balance Sheet table from a Company filing (10-Q, 10-K) using the filing url. Edgar nicely provides just the table, which can be easily extracted using Pandas for further analysis. We want to convert the statement to a common size statement by dividing each year by the Total Equity amount. As a result, we can see each caption relative to the overall Total Equity.

## Imports

In [1]:
from Helpers import edgar_support
import re

## Set Up

Get the URL for the edgar filing. This is the "index-htm" page edgar directs you to when you click on a quarterly/annual report. We sould convert the url to redirect us to the folder structure of the filing package. Once inside, the folder has a file called the "FilingSummary.xml" which describes each item in the folder. We will scrape the filing summary to identify the file the statement is stored in. 

In [2]:
edgar_filing_url = "https://www.sec.gov/Archives/edgar/data/1617640/000161764021000012/0001617640-21-000012-index.html"
base_url, filing_summary_url = edgar_support.convert_filing_to_folder(edgar_filing_url=edgar_filing_url)
slugs = edgar_support.retrieve_face_report_slugs(filing_summary_url)
balance_sheet_url = base_url + "/" + slugs["BS"]

## Clean

In order to analyze the statement, we first use pandas to read the table form the URL we discovered above. We clean the dataframe by renaming the first column header,  converting the multi-index columns to be a single index, and converting the data types to be numeric. Next, we remove rows where all numeric columns are all missing.

Lastly, we return the common size statement by dividng by the Total Equity row.

In [24]:
bs = edgar_support.get_clean_table(balance_sheet_url)
bs = bs.pipe(edgar_support.remove_rows_of_zeros)

divisor_caption = [caption for caption in bs["Captions"] if re.search(r"total", caption.lower()) and re.search(r"equity", caption.lower())][0]
bs = edgar_support.common_size_financial_statement(bs, divisor=bs.query("Captions == @divisor_caption").select_dtypes("number").iloc[0])

Unable to clean column - Captions.


## Visualize

We use pandas styler objects to add padding, add background colors, and format our numbers.

In [26]:
bs.style\
    .background_gradient(cmap='Blues', axis=0,)\
    .set_properties(**{'padding': '10px'})\
    .format({col: '{:,.2%}'.format for col in bs.select_dtypes("number").columns })\


,Captions,"Dec. 31, 2020","Dec. 31, 2019"
1,Cash and cash equivalents,35.92%,33.22%
2,Short-term investments,46.78%,37.29%
3,"Accounts receivable, net of allowance for doubtful accounts of $3,427 and $4,522 at December 31, 2020 and 2019, respectively",1.47%,1.95%
4,Mortgage loans held for sale,6.98%,1.06%
5,Inventory,10.36%,24.35%
6,Prepaid expenses and other current assets,1.60%,1.69%
7,Restricted cash,1.60%,2.61%
8,Total current assets,104.70%,102.18%
9,Contract cost assets,1.07%,1.32%
10,"Property and equipment, net",4.14%,4.96%
